In [1]:
from sympy import *
import sympy as sp
from sympy import I as j #import complex number as j
from sympy.solvers.solveset import linsolve, nonlinsolve
from sympy.parsing.latex import parse_latex
from sympy.parsing.sympy_parser import parse_expr
from IPython.display import display, Math, HTML
import ipywidgets as widgets

#for plotting
from numpy import linspace
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

try:
    import matlab.engine
    eng = matlab.engine.start_matlab()
except:
    display("MATLAB can't be loaded!")
    
#define symbols
Res = symbols("Res")
R=symbols("R_{0:7}", real = True)
I=symbols("I_{0:7}")
sI=symbols("i_{0:7}")
C=symbols("C_{0:7}", real=True)
L=symbols("L_{0:7}")
V=symbols("V_{0:7}")
Vin = symbols("V_in")
Vr = symbols("V_i*n")
t = symbols("t", positive=True)
s = symbols("s")
a = symbols("a_0:9", real = True)
b = symbols("b")
omega=symbols("omega")
#j=symbols("j")
Pr3 = symbols("P_R3")

#udeful SI prefix dict
prefix = {'y': 1e-24,  # yocto
           'z': 1e-21,  # zepto
           'a': 1e-18,  # atto
           'f': 1e-15,  # femto
           'p': 1e-12,  # pico
           'n': 1e-9,   # nano
           'u': 1e-6,   # micro
           'm': 1e-3,   # mili
           'c': 1e-2,   # centi
           'd': 1e-1,   # deci
           '' : 1,      # nothing
           'k': 1e3,    # kilo
           'M': 1e6,    # mega
           'G': 1e9,    # giga
           'T': 1e12,   # tera
           'P': 1e15,   # peta
           'E': 1e18,   # exa
           'Z': 1e21,   # zetta
           'Y': 1e24,   # yotta
    }

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
outIPy = widgets.Output(layout={'border': '2px solid black'})
initNumOfBoxes = {"new" : 2}
numOfR = 5
textBox = []
subsEqNum = 1

prsEq = []
linEq = []
lapEq = []
cEq = []
#---- plotting
mpl.rcParams['text.usetex'] = True
#amsmath needed for \text in equations and bm for \bm bold text
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}\usepackage{bm}\usepackage{gensymb}'

#floor with precision
def my_floor(a, precision=0):
    return np.true_divide(np.floor(a * 10**precision), 10**precision)
#ceil with precision
def my_ceil(a, precision=0):
    return np.true_divide(np.ceil(a * 10**precision), 10**precision)


@outIPy.capture()  #capture the plot output
def prettyPlot(expr, x_start, x_end, x_inc, si_prefix, x_title="", y_title="", x_unit="", y_label="", legend="True", logScale="True"):
    global lam_x, x_vals, y_vals, symExpr
    x_inc = int(x_inc)
    
    #display(x_start)
    #display(x_end)
    
    def diracDeltaArray(x):
        #resultArray = []
        #for i, val in np.ndenumerate(x):
        #    resultArray.append(DiracDelta(val))

        #return resultArray     
        if type(x) == int:
            return 0
        else:
            return np.zeros(len(x))
            
    fig = plt.figure(figsize=(8, 4), dpi=300)
    ax = plt.gca()
 
    
    symExpr = expr #atan((740*pi)/(1000+R))
    x_symb = list(symExpr.free_symbols)
    
    if len(x_symb) > 1:
        display("More than two symbols in equation")
        return 1
    else:
        x_symb = x_symb[0]
    
    symExpr = parse_expr(str(symExpr.subs("e", E)).replace("theta", "Heaviside").replace("delta", "DiracDelta")) #theta does'nt plot properly

    #display(symExpr)
    
    #np.seterr(all="ignore")
    
    
    
    #vect_x = np.vectorize(lam_x)
    
    
    
    if logScale == True:
        ax.set_xscale('log')
        #ax.tick_params(which="major",bottom=True,left=True, direction='out')
        ax.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
        ax.tick_params(which="minor",bottom=True,left=False, direction='in', length=5, width=1)
        x_vals = np.logspace(np.log10(x_start), np.log10(x_end), num=x_inc) #do log spacing for log axis
    else:
        x_vals = np.linspace(x_start, x_end, x_inc)
        #set x and y minor tick locations
        ax.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
        ax.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
        
    if si_prefix == "2pi":
        #x and y calculation mode for angular velocity
        lam_x = lambdify(x_symb, symExpr, modules=[{"DiracDelta": diracDeltaArray}, 'numpy'])
        y_vals = lam_x(x_vals)
        x_vals = x_vals/(2*np.pi)
        si_prefix = ""
        x_symb = symbols("f")
        plt.xlabel("$" + x_title + latex(x_symb) + ", " + si_prefix + x_unit + "$")
    else:
        #normal x and y calculation mode
        if checkBox4.value == True:
            display("compatability mode:")
            x_vals = np.linspace(x_start, x_end, x_inc)
            y_vals = np.array([])
            for i in x_vals:
                y_vals = np.append(y_vals, symExpr.subs("t", i*prefix[si_prefix])) #maybe t doesn't need quotes 
            
            y_vals = np.real(y_vals)
        else:
            lam_x = lambdify(x_symb, symExpr, modules=[{"DiracDelta": diracDeltaArray}, 'numpy'])
            y_vals = lam_x(x_vals*prefix[si_prefix])
        
        plt.xlabel("$" + x_unit + "$")
        
    plt.plot(x_vals, y_vals)
    
    #plt.ylabel(y_title + latex(y_symb) + latex(", " + si_prefix) + x_unit)
    plt.ylabel(r"$" + y_label + "$")
    #plt.ylabel(r"Fāze $\bm{\varphi}$, °")

    #plt.yticks(np.arange(my_floor(min(y_vals), -1), my_ceil(max(y_vals), -1)+1, 10)) may bee needed for some functions
    
    if legend == True:
        leg = plt.legend(['$' + latex(symExpr) + '$'], framealpha = 1, prop={'size': 12})
    sns.set_style("whitegrid")
    sns.set_context("talk")

    
        
    ax.grid(b=True, which='major', color='dimgrey', linewidth=2.0)
    ax.grid(b=True , which='minor', axis='both', color='grey', linestyle='dotted', linewidth=0.5)
    plt.show()
     
# plotting ---


style = {'description_width': 'initial'}
layout = widgets.Layout(width='auto', height='40px')

def drawTBox(change): #dinamically draw prsEq input text boxes
    out.clear_output()
    
    global numOfBoxes   #global needed because without it the next statement wouldn't update the global variable
    numOfBoxes = int(change["new"])
    
    global textBox
    textBox = []
    for i in range(numOfBoxes):
        textString = "Eq" + str(i+1) + ":"
        textBox.append(widgets.Text(description=textString))
        #print(numOfBoxes)
        with out:
            display(textBox[i])

drawTBox(initNumOfBoxes)

dropdown = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value = str(initNumOfBoxes["new"]),
    description='Number of prsEqs:',
    disabled=False,
    style=style
)

#def tes(change):
#    print(change["new"])

def changeSubs(change):  #change for which prsEqs to substitute values
    global subsEqNum
    subsEqNum = change["new"]

#drawTBox(int(dropdown.value))

radioButton = widgets.RadioButtons(
    options=[('parsed equation',1), ('linsolve equation',2), ('custom equation',3)],
    disabled=False,
    layout=layout
)

radioLabel = widgets.Label('Substitute or inv. laplace transform:', layout=widgets.Layout(width='auto'))

checkBox1 = widgets.Checkbox(
    value=False,
    description='Legend',
    indent=False,
    layout = widgets.Layout(width='100px')
)

checkBox2 = widgets.Checkbox(
    value=False,
    description='LogScale',
    indent=False,
    layout = widgets.Layout(width='100px')
)

checkBox3 = widgets.Checkbox(
    value=False,
    description='LongLaplace',
    indent=False,
    layout = widgets.Layout(width='100px')
)

checkBox4 = widgets.Checkbox(
    value=False,
    description='Larger eq CompMode',
    indent=False,
)

checkBox5 = widgets.Checkbox(
    value=False,
    description='MATLAB mode',
    indent=False,
    layout = widgets.Layout(width='150px')
)

button1 = widgets.Button(
    description='Parse equations',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

button2 = widgets.Button(
    description='LinSolve for I',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

button3 = widgets.Button(
    description='Substitute in equations',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button4 = widgets.Button(
    description='Clear Subs Val',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button5 = widgets.Button(
    description='Clear Output',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button6 = widgets.Button(
    description='Inverse Laplace Trans',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button7 = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button8 = widgets.Button(
    description='Graph',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button9 = widgets.Button(
    description='Solve for V',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button10 = widgets.Button(
    description='Eval',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

textBox1 = widgets.Text(
    placeholder="x unit",
    layout = widgets.Layout(width='100px')
)

textBox2 = widgets.Text(
    placeholder="y text", 
    layout = widgets.Layout(width='100px')
)

textBox3 = widgets.Text(
    placeholder="x-start", 
    layout = widgets.Layout(width='100px')
)

textBox4 = widgets.Text(
    placeholder="end-x", 
    layout = widgets.Layout(width='100px')
)

textBox5 = widgets.Text(
    placeholder="prefix", 
    layout = widgets.Layout(width='100px')
)

#prsEq = [Eq((R[1]+1/(C[1]*s))*I[1]-(1/(C[1]*s))*I[2],V[1]),
#           Eq(-1/(C[1]*s)*I[1]+(R[2]+1/(C[1]*s))*I[2],0)]
#display(prsEq[0])
#for i in prsEq:
 #   display(i)
#(R[1]+1/(C[1]*s))*I[1]-(1/(C[1]*s))*I[2]

#def handle_submit(sender):
 #   print(textBox.value)
  #  print(sender)

def handle_button8(a): # graph equations
    boxInLhs = customEqBox.value
    if customEqBox.value:
        
        for key, val in {"prsEq":prsEq, "linEq":linEq, "lapEq":lapEq, "cEq":cEq}.items():
            
            for num in range(len(val)):
        
                boxInLhsNew = boxInLhs.find(key + "[" + str(num) + "]")
                
                
                if boxInLhsNew >= 0:
                    
                    boxInLhsNew = val[num]
                    
                    break
            else:
                continue
            break
        
        
        prefix_value = textBox5.value
        
        if textBox3.value == "" or textBox4.value == "":
            start_val = 0
            end_val = 1
            stepSize = 1e4 
        elif textBox5.value == "2pi" or textBox5.value == "2*pi":
            start_val = float(textBox3.value)*2*np.pi  #sympy pi dosent work!!!
            end_val = float(textBox4.value)*2*np.pi  #sympy pi dosent work!!!
            prefix_value = '2pi'
            stepSize = 1e4
        else:
            start_val = float(textBox3.value)
            end_val = float(textBox4.value)
            stepSize = 1e4 
        
        prettyPlot(boxInLhsNew.rhs, start_val, end_val, stepSize, si_prefix=prefix_value, x_unit=textBox1.value, y_label=textBox2.value, legend=checkBox1.value, logScale=checkBox2.value)

    
def handle_button7(but10=False):  #custom equation calc
    global cEq
    cEq = []
    boxInLhs = customEqBox.value
    boxInRhs = boxInLhs
    
    if customEqBox.value:
        
        for key, val in {"prsEq":prsEq, "linEq":linEq, "lapEq":lapEq}.items():
            
            for num in range(len(val)):
                
                if str(type(val[num])) != "<class 'sympy.core.relational.Equality'>":
                    val[num] = Eq(Res, val[num])
                
                prsEqLhs = str(val[num].lhs).replace("{", "").replace("}", " ") #because parse_expre dosen't like I_{x}
                prsEqRhs = str(val[num].rhs).replace("{", "").replace("}", " ")
                boxInLhs = boxInLhs.replace(key + "[" + str(num) + "]", prsEqLhs)
                boxInRhs = boxInRhs.replace(key + "[" + str(num) + "]", prsEqRhs)
            
        boxInLhs = parse_expr(boxInLhs).subs("j", j)
        boxInRhs = parse_expr(boxInRhs).subs("j", j)
        cEq.append(Eq(boxInLhs, boxInRhs))
    
    #elif boxIn.find("linEq[")
    #for n in [prsEq] 
    
    if but10: 
        for eq in cEq:
            outIPy.append_display_data(eq.evalf())
    else:
        for eq in cEq:
            outIPy.append_display_data(eq)

def handle_button10(a):
    handle_button7(but10=True)

def matInvLaplace(eq):  #calculate inverse laplace transform with MATLAB
    with open("reservedForInvLaplace.m","w+") as f:
        f.write("syms s\n" \
                "invLaplaceInput = " + eq + ";")
        
    eng.reservedForInvLaplace(nargout=0)
    eng.reservedForInvLaplace(nargout=0)
    return eng.matInvLaplace(eng.workspace['invLaplaceInput'])
    #print(eng.workspace['reslt'])
    
def handle_button6(someRandomVal):  #laplace transform the solved linEqs 
    global lapEq
    lapEq = []
    if subsEqNum == 1: #for parsed prsEqs
        eqNum = 0

        #print(subsEqNum)
    elif subsEqNum == 2: #for linsolve linEqs
        eqNum = 1

    elif subsEqNum == 3: #for custom equations
        eqNum = 2
    
    
    for n, res in enumerate([prsEq, linEq, cEq][eqNum]):
        
        if (checkBox3.value == True) and (checkBox5.value == True):
            outIPy.append_display_data("Both laplace checkboxes can't be checked at the same time.")
            return -1

        elif checkBox5.value == True: #calculate with matlab
            lapTrans = matInvLaplace(str(res.rhs).replace("**", "^")).replace("heaviside", "Heaviside") 
            #sympy dosent uderstand heaviside with a small h 

        elif checkBox3.value == True: 
            outIPy.append_display_data("doing long laplace transform:")
            #replace very small numbers with symbols
            equa = str(res.rhs)
            eoffset = 2
            offset = 1
            prevELoc = 0
            maxPower = -4
            smallNum = []
            eloc = equa.find("e-", prevELoc)
            while (eloc) > -1:  #walrus dosent work in python 3.7 ->[ :=  ]
                eloc = equa.find("e-", prevELoc) # when you use walrus you don't need this line
                prevELoc = eloc + 2 
                
                #display(equa[eloc+offset:eloc+offset+1])
                while equa[eloc+eoffset:eloc+eoffset+1].isdigit():
                    #display(equa[eloc+offset:eloc+offset+1])
                    eoffset += 1

                if int(equa[eloc+1:eloc+eoffset]) < maxPower:
                    offset = 1
                    while equa[eloc-offset:eloc-offset+1].isdigit() or equa[eloc-offset:eloc-offset+1] == ".":
                        offset += 1
                    offset -= 1
                    #equa = equa[:eloc-offset] + equa[eloc+eoffset]
                    smallNum.append(equa[eloc-offset:eloc+eoffset])
                eoffset = 2

            for i, num in enumerate(smallNum):
                equa = equa.replace(num, "a_" + str(i) + "")

            equa = sp.parse_expr(equa)

            for i in range(len(smallNum)):
                equa = equa.subs("a_" + str(i) , a[i])

            
            lapTrans = inverse_laplace_transform(equa,s,t)
            #display(lapTrans)
            for i, num in enumerate(smallNum):
                lapTrans = lapTrans.subs(a[i], sp.S(num))


        else: #if no checkbox calculate normally
            lapTrans = inverse_laplace_transform(res.rhs,s,t)
        
            
        #display(lapTrans)
        
        lapTrans = simplify(lapTrans) # returns better results and also converts the matlab string output to sympy
        lapEq.append(Eq(sI[n+1], lapTrans)) #inv laplace dosent work for equalities
        
        outIPy.append_display_data(lapEq[n])


def handle_button5(a):  #clear output
    outIPy.clear_output()
    
#def handle_button4(a): #clear substitute input field values

def handle_button3(a):  #substitute values parser
    global prsEq
    if subsEqNum == 3:
        forRange = range(len(cEq))
    else:
        forRange = range(len(prsEq))
        
    for nEq in forRange:
        for key, textField in valueItems.items():
            for num, textF in enumerate(textField):
                if textF.value != "":
                    
                    #uncomment if you want to have prefixes added to input values
                    #if key=="L":
                    #    mult = 10**-3
                    #elif key=="C":
                    #    mult = 10**-6 
                    #else:
                    mult = 1
                    #print(key + "_{" + str(n) + "}")
                    #print(int(textF.value))
                    
                    textF = parse_expr(textF.value)
                    textF = textF.subs("j", j)  #this changes the symbol j to the imaginary number j
                    
                    bnum = "_{" + str(num) + "}" # because Vin and custom eqs dosent need the braces {}
                    if key=="Vin":
                        bnum = "_in"
                        num = "in"
                        key = "V"
                    
                    if key=="s":
                        bnum = ""
                        key = "s"
                        
                    if subsEqNum == 1: #for parsed prsEqs
                        prsEq[nEq] = prsEq[nEq].subs(key  + str(bnum), textF*mult)
                        
                        #print(subsEqNum)
                    elif subsEqNum == 2: #for linsolve linEqs
                        linEq[nEq] = linEq[nEq].subs(key + str(bnum), textF*mult)
                        
                    elif subsEqNum == 3: #for custom equations
                        cEq[nEq] = cEq[nEq].subs(key + "_" + str(num), textF*mult)
                        display()
                        #print(subsEqNum)
            
        if subsEqNum == 1: #for parsed prsEqs
            outIPy.append_display_data(prsEq[nEq])
        elif subsEqNum == 2: #for linsolve linEqs
            outIPy.append_display_data(linEq[nEq])
        elif subsEqNum == 3: #for custom equations
            outIPy.append_display_data(cEq[nEq])
                    #print(eq)
    #display(prsEq[0])

def handle_button9(a):  #linear solver for V
    vList = []
    global linEq
    for n, eq in enumerate(prsEq):
        vList.append(V[n+1])
        
    linEq = linsolve(prsEq,vList).args[0]
    linEq = list(linEq)  #needed as list in other functions
    #outIPy.append_display_data(type(linEq))
    outIPy.append_display_data("result:")
    for n, res in enumerate(linEq):
        linEq[n] = Eq(V[n+1], linEq[n])
        outIPy.append_display_data(Math(latex(linEq[n]))) #pretty print
    
def handle_button2(a):  #linear solver
    iList = []
    global linEq
    for n, eq in enumerate(prsEq):
        iList.append(I[n+1])
        
    linEq = linsolve(prsEq,iList).args[0]
    linEq = list(linEq)  #needed as list in other functions
    #outIPy.append_display_data(type(linEq))
    outIPy.append_display_data("result:")
    for n, res in enumerate(linEq):
        linEq[n] = Eq(I[n+1], linEq[n])
        outIPy.append_display_data(Math(latex(linEq[n]))) #pretty print
    
    
def handle_button1(a):  #input parser
    global prsEq
    latex_string = []
    prsEq = []

    for i in range(numOfBoxes):
        lstring = textBox[i].value
        lstring = lstring.replace(r"\ ", "") # remove blank spaces
        lstring = lstring.replace("log", r"\log") # for log to be correctly interpreted
        lstring = lstring.replace("ln", r"\ln") # for ln to be correctly interpreted
        lstring = lstring.replace("[", "{").replace("]", "}") # replace square brackets, because the interpreter dsnt understand them
        lstring = parse_latex(lstring)
        #lstring = lstring.subs(j, i)
        lstring = lstring.subs("V_{i*n}", Vin) # multiple charachter subscripts are parsed incorectly
        lstring = lstring.subs("e", E)
        lstring = lstring.subs("j", j)  #this changes the symbol j to the imaginary number j 
        prsEq.append(lstring)
        outIPy.append_display_data(prsEq[i])
        #display(type(prsEq[i]))
    prsEq = list(prsEq)
        
valueItems = {"R": [], "C": [], "L": [], "V": [],"Vin": [], "s": [], "I": []}
valueItemPlaceholder = {"R": "Ohms", "C": "F", "L": "H", "V": "Volts","Vin": "input eq","s": "input eq", "I": "Amps"}
valueChildList = []
for key in valueItems:
    if key=="Vin" or key=="s":
        oldNumOfR = numOfR
        numOfR = 1
    for n in range(numOfR):
        if key=="Vin" :
            n = "{in}"
            valueItems[key].append(widgets.Text(description="$" + "V" + "_" + str(n) + "$", placeholder=valueItemPlaceholder[key]))
        elif key=="s":
            valueItems[key].append(widgets.Text(description="$s$", placeholder=valueItemPlaceholder[key]))
        else:
            valueItems[key].append(widgets.Text(description="$" + key + "_" + str(n) + "$", placeholder=valueItemPlaceholder[key]))
    valueChildList.append(widgets.VBox(valueItems[key]))
    if key=="Vin" or key=="s":
        numOfR = oldNumOfR
    
box_layout = widgets.Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='500px',
                    height='',
                    flex_flow='column',
                    display='flex')
#carousel = widgets.Box(children=valueItems, layout=box_layout)
#carousel = widgets.Box(children=valueItems)

display(dropdown)
dropdown.observe(drawTBox, names="value")

display(widgets.Box(valueChildList))

display(out)
display(widgets.HBox([button1,button2,button9,button4,button5]))

display("Write custom expression:  prsEq[x] parsed   linEq[x] system of equation results   lapEq[x] inv. laplace results")
display("cEq[x] custom eq (only for graphing)  !!You cannot graph equations!! Just use basic selectors e.g. linEq[] or cEq[0] ")

customEqBox = widgets.Text(placeholder="Equation")
display(widgets.HBox([customEqBox, button7, button8, button10, checkBox1, checkBox2,textBox1,textBox2]))
display(widgets.HBox([textBox3,textBox4,textBox5, checkBox4]))
display(widgets.HBox([widgets.Text(placeholder="Paste here <<Ctrl+v>>"),widgets.Text(placeholder="or here <<Ctrl+v>>"),widgets.Text(placeholder="or even here <<Ctrl+v>>")]))

display(widgets.HBox([radioLabel, radioButton, button6 ,button3, checkBox3, checkBox5], layout=widgets.Layout(padding = "0px 0px 20px 0px")))
radioButton.observe(changeSubs, names="value")

display(outIPy)

button1.on_click(handle_button1)
button2.on_click(handle_button2)
button3.on_click(handle_button3)
#button4.on_click(handle_button4)
button5.on_click(handle_button5)
button6.on_click(handle_button6)
button7.on_click(handle_button7)
button8.on_click(handle_button8)
button9.on_click(handle_button9)
button10.on_click(handle_button10)

Dropdown(description='Number of pEqs:', index=1, options=('1', '2', '3', '4'), style=DescriptionStyle(descript…

Box(children=(VBox(children=(Text(value='', description='$R_0$', placeholder='Ohms'), Text(value='', descripti…

Output(layout=Layout(border='1px solid black'))

'Write custom expression:  pEq[x] parsed   linEq[x] system of equation results   lapEq[x] inv. laplace results'

Output(layout=Layout(border='2px solid black'))